## Import Libraries ##

In [3]:
import os
import numpy as np
import pandas as pd
import rasterio
from osgeo import gdal, osr
import geopandas as gpd
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tqdm import tqdm
from rasterio.features import geometry_window, shapes, geometry_mask
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling

## Define Filepaths ##

In [4]:
BASE_DIR = 'C:\\Users\\chris\\Desktop\\use-of-AI-to-eradicate-extreme-poverty'
SETTLEMENT_DIRS = [
    os.path.join(BASE_DIR, 'data/GlobalHumanSettlement/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0_R11_C22.tif'),
    os.path.join(BASE_DIR, 'data/GlobalHumanSettlement/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0_R12_C22.tif')
]
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data', 'nightlights', 'viirs_2020_00N060W.tif')
ROADS_DIR = os.path.join(BASE_DIR, 'data', 'roads', 'GRIP4_density_total','grip4_total_dens_m_km2.asc')
HEALTH_DIR = os.path.join(BASE_DIR, 'data', 'health_facilities', 'healthcare_2020.shp')
LSMS_DIR = os.path.join(BASE_DIR, 'data', 'LSMS_2019')
csv_file_path = os.path.join(LSMS_DIR, 'HouseholdGeovariables_csv', 'householdgeovariables_ihs5.csv')

## Modeling ##

## splitted model script ##

## part 1 ##

In [5]:
import os
import numpy as np
import rasterio
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from tqdm import tqdm
import joblib

def load_raster(path):
    with rasterio.open(path) as src:
        data = src.read(1)
        nodata = src.nodata
        return np.where(data == nodata, np.nan, data), src.profile

def process_in_chunks(paths, chunk_size=10000):
    data_list = []
    labels_list = []
    positions_list = []

    with rasterio.open(paths[0]) as src:
        height, width = src.shape
        total_pixels = height * width
        for i in tqdm(range(0, total_pixels, chunk_size), desc="Processing chunks"):
            row_start = i // width
            col_start = i % width
            row_end = row_start + (chunk_size // width)
            col_end = col_start + chunk_size % width
            
            if col_end > width:
                col_end = width

            window = rasterio.windows.Window(col_start, row_start, col_end - col_start, row_end - row_start)

            chunk_data = []
            for path in paths:
                with rasterio.open(path) as src:
                    chunk = src.read(1, window=window)
                    chunk_nodata = src.nodata
                    chunk_data.append(np.where(chunk == chunk_nodata, np.nan, chunk).flatten())

            valid_mask_chunk = ~np.isnan(chunk_data[0])
            for data in chunk_data[1:]:
                valid_mask_chunk &= ~np.isnan(data)

            if np.any(valid_mask_chunk):
                data_list.append(np.column_stack([data[valid_mask_chunk] for data in chunk_data[:-1]]))
                labels_list.append(chunk_data[-1][valid_mask_chunk])
                row_col_indices = np.column_stack(np.where(valid_mask_chunk.reshape(window.height, window.width)))
                global_indices = row_col_indices + np.array([row_start, col_start])
                positions_list.append(global_indices)

    return np.vstack(data_list), np.concatenate(labels_list), np.vstack(positions_list), height, width

BASE_DIR = r'C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty'
model_results_dir = os.path.join(BASE_DIR, 'results', 'model')
os.makedirs(model_results_dir, exist_ok=True)

paths = [
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_nightlights.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_settlements.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_roads_density.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_healthcare.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'interpolation_gap_poor.tif')
]

print("Loading and processing data...")
features, labels, positions, height, width = process_in_chunks(paths)

print("Handling missing values...")
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

print("Scaling features...")
scaler = RobustScaler()
features_scaled = scaler.fit_transform(features_imputed)

print("Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

print("Performing grid search...")
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Save the model
model_save_path = os.path.join(model_results_dir, 'best_model.joblib')
joblib.dump(best_model, model_save_path)
print(f"Model saved to {model_save_path}")

# Print best parameters and intermediate results
print("Best parameters found: ", grid_search.best_params_)

print("Predicting and evaluating...")
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print("Evaluation Metrics:")
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R² Score:", r2)

Loading and processing data...


Processing chunks: 100%|███████████████████████████████████████████████████████████| 3200/3200 [01:27<00:00, 36.41it/s]


Handling missing values...
Scaling features...
Splitting dataset...
Performing grid search...
Fitting 2 folds for each of 8 candidates, totalling 16 fits
Model saved to C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty\results\model\best_model.joblib
Best parameters found:  {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 100}
Predicting and evaluating...
Evaluation Metrics:
Mean Squared Error: 9.311295890005903
Root Mean Squared Error: 3.0514416084870284
R² Score: 0.8254775970704676


## part 2 ##

In [6]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.warp import Resampling
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
import joblib
from tqdm import tqdm

def load_raster(path):
    with rasterio.open(path) as src:
        data = src.read(1)
        nodata = src.nodata
        return np.where(data == nodata, np.nan, data), src.profile

def process_in_overlapping_chunks(paths, height, width, chunk_size=100, overlap=10):
    data_list = []
    positions_list = []

    step_size = chunk_size - overlap

    for row in tqdm(range(0, height, step_size), desc="Processing rows"):
        for col in range(0, width, step_size):
            row_end = min(row + chunk_size, height)
            col_end = min(col + chunk_size, width)
            
            window = Window(col, row, col_end - col, row_end - row)

            chunk_data = []
            for path in paths:
                with rasterio.open(path) as src:
                    chunk = src.read(1, window=window)
                    chunk_nodata = src.nodata
                    chunk_data.append(np.where(chunk == chunk_nodata, np.nan, chunk).flatten())

            valid_mask_chunk = ~np.isnan(chunk_data[0])
            for data in chunk_data[1:]:
                valid_mask_chunk &= ~np.isnan(data)

            if np.any(valid_mask_chunk):
                data_list.append(np.column_stack([data[valid_mask_chunk] for data in chunk_data]))
                row_col_indices = np.column_stack(np.where(valid_mask_chunk.reshape(window.height, window.width)))
                global_indices = row_col_indices + np.array([row, col])
                positions_list.append(global_indices)

    return np.vstack(data_list), np.vstack(positions_list), height, width

def resample_raster(data, profile, new_resolution):
    """Resample raster data to a new resolution using the given scale factor."""
    scale_factor = new_resolution / profile['transform'][0]  # new_resolution / original pixel size in meters
    new_height = int(data.shape[0] / scale_factor)
    new_width = int(data.shape[1] / scale_factor)

    new_data = np.empty((new_height, new_width), dtype=np.float32)
    new_transform = profile['transform'] * profile['transform'].scale(scale_factor, scale_factor)

    with rasterio.open(
        '/vsimem/temp.tif', 'w',
        driver='GTiff',
        height=new_height,
        width=new_width,
        count=1,
        dtype='float32',
        crs=profile['crs'],
        transform=new_transform,
    ) as dst:
        dst.write(data, 1)

    with rasterio.open('/vsimem/temp.tif') as src:
        resampled_data = src.read(
            1,
            out_shape=(1, new_height, new_width),
            resampling=Resampling.bilinear,
        )
        resampled_profile = src.profile

    return resampled_data, resampled_profile

BASE_DIR = r'C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty'
model_results_dir = os.path.join(BASE_DIR, 'results', 'model')
model_path = os.path.join(model_results_dir, 'best_model.joblib')

paths = [
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_nightlights.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_settlements.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_roads_density.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_healthcare.tif')
]

print("Loading and processing data...")
with rasterio.open(paths[0]) as src:
    height, width = src.shape

features, positions, height, width = process_in_overlapping_chunks(paths, height, width)

print(f"Total features shape: {features.shape}")
print(f"Total positions shape: {positions.shape}")
print(f"Raster dimensions: height={height}, width={width}")

print("Handling missing values...")
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)
print("Missing values imputed.")

print("Scaling features...")
scaler = RobustScaler()
features_scaled = scaler.fit_transform(features_imputed)
print("Features scaled.")

print("Loading the saved model...")
best_model = joblib.load(model_path)

# Prepare full predictions array
full_predictions = np.full((height, width), np.nan)

print("Predicting values...")
flat_predictions = best_model.predict(features_scaled)
for (row, col), pred in zip(positions, flat_predictions):
    full_predictions[row, col] = pred

# Save the non-resampled predictions raster
non_resampled_raster_path = os.path.join(BASE_DIR, 'results', 'rasters', 'predicted_gap_poor.tif')
target_raster_path = os.path.join(BASE_DIR, 'results', 'rasters', 'interpolation_gap_poor.tif')
with rasterio.open(target_raster_path) as src:
    profile_target = src.profile

profile_target['dtype'] = 'float32'
profile_target['nodata'] = np.nan
with rasterio.open(non_resampled_raster_path, 'w', **profile_target) as dst:
    dst.write(full_predictions, 1)

print(f"Non-resampled predictions raster saved at {non_resampled_raster_path}")

# Load the actual raster
actual_values, profile_actual = load_raster(target_raster_path)

# Resample the actual and predicted rasters to 8km x 8km resolution
new_resolution = 8000
resampled_actual, profile_actual_resampled = resample_raster(actual_values, profile_target, new_resolution)
resampled_predicted, profile_predicted_resampled = resample_raster(full_predictions, profile_target, new_resolution)

# Save the resampled rasters
resampled_actual_path = os.path.join(BASE_DIR, 'results', 'rasters', 'actual_gap_poor_8kmRes.tif')
resampled_predicted_path = os.path.join(BASE_DIR, 'results', 'rasters', 'predicted_gap_poor_8kmRes.tif')
with rasterio.open(resampled_actual_path, 'w', **profile_actual_resampled) as dst:
    dst.write(resampled_actual, 1)

with rasterio.open(resampled_predicted_path, 'w', **profile_predicted_resampled) as dst:
    dst.write(resampled_predicted, 1)

print(f"Resampled rasters saved at {resampled_actual_path} and {resampled_predicted_path}")

Loading and processing data...


Processing rows: 100%|█████████████████████████████████████████████████████████████████| 99/99 [02:11<00:00,  1.33s/it]


Total features shape: (27482120, 4)
Total positions shape: (27482120, 2)
Raster dimensions: height=8871, width=3607
Handling missing values...
Missing values imputed.
Scaling features...
Features scaled.
Loading the saved model...
Predicting values...
Non-resampled predictions raster saved at C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty\results\rasters\predicted_gap_poor.tif
Resampled rasters saved at C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty\results\rasters\actual_gap_poor_8kmRes.tif and C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty\results\rasters\predicted_gap_poor_8kmRes.tif


## part 3 ##

In [7]:
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import joblib

# Define paths
BASE_DIR = r'C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty'
model_results_dir = os.path.join(BASE_DIR, 'results', 'model')
predicted_raster_path = os.path.join(BASE_DIR, 'results', 'rasters', 'predicted_gap_poor.tif')
actual_raster_path = os.path.join(BASE_DIR, 'results', 'rasters', 'interpolation_gap_poor.tif')

# Load the actual and predicted rasters
def load_raster(path):
    with rasterio.open(path) as src:
        data = src.read(1)
        nodata = src.nodata
        return np.where(data == nodata, np.nan, data), src.profile

def process_in_chunks(paths, chunk_size=10000):
    data_list = []
    labels_list = []
    positions_list = []

    with rasterio.open(paths[0]) as src:
        height, width = src.shape
        total_pixels = height * width
        for i in tqdm(range(0, total_pixels, chunk_size), desc="Processing chunks"):
            row_start = i // width
            col_start = i % width
            row_end = row_start + (chunk_size // width)
            col_end = col_start + chunk_size % width

            if col_end > width:
                col_end = width

            window = rasterio.windows.Window(col_start, row_start, col_end - col_start, row_end - row_start)

            chunk_data = []
            for path in paths:
                with rasterio.open(path) as src:
                    chunk = src.read(1, window=window)
                    chunk_nodata = src.nodata
                    chunk_data.append(np.where(chunk == chunk_nodata, np.nan, chunk).flatten())

            valid_mask_chunk = ~np.isnan(chunk_data[0])
            for data in chunk_data[1:]:
                valid_mask_chunk &= ~np.isnan(data)

            if np.any(valid_mask_chunk):
                data_list.append(np.column_stack([data[valid_mask_chunk] for data in chunk_data[:-1]]))
                labels_list.append(chunk_data[-1][valid_mask_chunk])
                row_col_indices = np.column_stack(np.where(valid_mask_chunk.reshape(window.height, window.width)))
                global_indices = row_col_indices + np.array([row_start, col_start])
                positions_list.append(global_indices)

    return np.vstack(data_list), np.concatenate(labels_list), np.vstack(positions_list), height, width

print("Loading and processing data...")
paths = [
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_nightlights.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_settlements.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_roads_density.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'aligned_healthcare.tif'),
    os.path.join(BASE_DIR, 'results', 'rasters', 'interpolation_gap_poor.tif')
]
features, labels, positions, height, width = process_in_chunks(paths)

print("Handling missing values...")
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

print("Scaling features...")
scaler = RobustScaler()
features_scaled = scaler.fit_transform(features_imputed)

# Load the model
model_path = os.path.join(model_results_dir, 'best_model.joblib')
best_model = joblib.load(model_path)

# Train-Test Evaluation
print("Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

print("Predicting and evaluating on train-test split...")
predictions_test = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, predictions_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, predictions_test)

print("Evaluation Metrics on Train-Test Split:")
print("Mean Squared Error:", mse_test)
print("Root Mean Squared Error:", rmse_test)
print("R² Score:", r2_test)

# Plotting actual vs predicted for train-test split
plt.scatter(y_test, predictions_test, alpha=0.3)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted (Train-Test Split)')
plt.grid(True)
plt.savefig(os.path.join(model_results_dir, 'actual_vs_predicted_train_test.png'))
plt.close()

# Plotting residuals for train-test split
residuals_test = y_test - predictions_test
plt.hist(residuals_test, bins=50, alpha=0.75)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals (Train-Test Split)')
plt.grid(True)
plt.savefig(os.path.join(model_results_dir, 'residuals_histogram_train_test.png'))
plt.close()

# Plot distribution of actual vs predicted for train-test split
plt.hist(y_test, bins=50, alpha=0.5, label='Actual Values', color='blue')
plt.hist(predictions_test, bins=50, alpha=0.5, label='Predicted Values', color='orange')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Distribution of Actual and Predicted Values (Train-Test Split)')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(model_results_dir, 'value_distributions_train_test.png'))
plt.close()

# Full Raster Prediction
print("Predicting on full raster...")
flat_predictions = best_model.predict(features_scaled)
full_predictions = np.full((height, width), np.nan)
for (row, col), pred in zip(positions, flat_predictions):
    full_predictions[row, col] = pred

# Save the full predictions raster
profile_target = load_raster(actual_raster_path)[1]
predicted_raster_path = os.path.join(BASE_DIR, 'results', 'rasters', 'predicted_gap_poor_full.tif')
profile_target['dtype'] = 'float32'
profile_target['nodata'] = np.nan
with rasterio.open(predicted_raster_path, 'w', **profile_target) as dst:
    dst.write(full_predictions, 1)

print(f"Full predictions raster saved at {predicted_raster_path}")

# Load the actual and full predicted rasters for evaluation
actual_values, profile_actual = load_raster(actual_raster_path)
predicted_values, profile_predicted = load_raster(predicted_raster_path)

# Flatten the arrays and remove NaNs for comparison
actual_flat = actual_values.flatten()
predicted_flat = predicted_values.flatten()

valid_mask = ~np.isnan(actual_flat) & ~np.isnan(predicted_flat)
actual_flat = actual_flat[valid_mask]
predicted_flat = predicted_flat[valid_mask]

# Calculate evaluation metrics based on the full raster
mse_full = mean_squared_error(actual_flat, predicted_flat)
rmse_full = np.sqrt(mse_full)
r2_full = r2_score(actual_flat, predicted_flat)

print("Evaluation Metrics on Full Raster:")
print("Mean Squared Error:", mse_full)
print("Root Mean Squared Error:", rmse_full)
print("R² Score:", r2_full)

# Plotting actual vs predicted (full raster)
plt.scatter(actual_flat, predicted_flat, alpha=0.3)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted (Full Raster)')
plt.grid(True)
plt.savefig(os.path.join(model_results_dir, 'actual_vs_predicted_full.png'))
plt.close()

# Plotting residuals (full raster)
residuals_full = actual_flat - predicted_flat
plt.hist(residuals_full, bins=50, alpha=0.75)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals (Full Raster)')
plt.grid(True)
plt.savefig(os.path.join(model_results_dir, 'residuals_histogram_full.png'))
plt.close()

# Plot distribution of actual vs predicted (full raster)
plt.hist(actual_flat, bins=50, alpha=0.5, label='Actual Values', color='blue')
plt.hist(predicted_flat, bins=50, alpha=0.5, label='Predicted Values', color='orange')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Distribution of Actual and Predicted Values (Full Raster)')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(model_results_dir, 'value_distributions_full.png'))
plt.close()

# Display the original and predicted rasters for comparison (full raster)
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
axs[0].imshow(actual_values, cmap='viridis', interpolation='nearest')
axs[0].set_title('Actual Gap Poor Values (Full Raster)')
axs[0].set_axis_off()
axs[1].imshow(predicted_values, cmap='viridis', interpolation='nearest')
axs[1].set_title('Predicted Gap Poor Values (Full Raster)')
axs[1].set_axis_off()
plt.savefig(os.path.join(model_results_dir, 'actual_predicted_comparison_full.png'))
plt.close()

# Print summary
print(f"Train-Test Evaluation Metrics: MSE={mse_test}, RMSE={rmse_test}, R²={r2_test}")
print(f"Full Raster Evaluation Metrics: MSE={mse_full}, RMSE={rmse_full}, R²={r2_full}")

Loading and processing data...


Processing chunks: 100%|███████████████████████████████████████████████████████████| 3200/3200 [01:13<00:00, 43.61it/s]


Handling missing values...
Scaling features...
Splitting dataset...
Predicting and evaluating on train-test split...
Evaluation Metrics on Train-Test Split:
Mean Squared Error: 9.311295890005903
Root Mean Squared Error: 3.0514416084870284
R² Score: 0.8254775970704676
Predicting on full raster...
Full predictions raster saved at C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty\results\rasters\predicted_gap_poor_full.tif
Evaluation Metrics on Full Raster:
Mean Squared Error: 8.883992
Root Mean Squared Error: 2.9806027
R² Score: 0.8337105884277722
Train-Test Evaluation Metrics: MSE=9.311295890005903, RMSE=3.0514416084870284, R²=0.8254775970704676
Full Raster Evaluation Metrics: MSE=8.883992195129395, RMSE=2.980602741241455, R²=0.8337105884277722
